In [5]:
import pandas as pd
import joblib

df = pd.read_csv("호선정리.csv", encoding='CP949')
list_data = df.values.tolist()

def get_station_number(station_name, station_data):
    for item in station_data:
        if item[2] == station_name:
            return int(item[0])
    return None

def determine_direction(start_station, end_station, station_data):
    start_num = get_station_number(start_station, station_data)
    end_num = get_station_number(end_station, station_data)
    return "상선" if (end_num - start_num) > 0 else "하선"

def compute_average_congestion(congestion_values):
    return sum(congestion_values) / len(congestion_values)

def print_congestion_predictions(start_station, end_station, input_YEAR, input_MONTH, input_DAY, input_LINE, input_TIME, station_data):
    filtered_station_data = [item for item in station_data if item[1] == input_LINE]
    stations = [item[2] for item in filtered_station_data]
    
    start_idx = stations.index(start_station)
    end_idx = stations.index(end_station)
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx

    direction = determine_direction(start_station, end_station, station_data)
    
    model = joblib.load(f'XGB_reg_num{input_LINE}.pkl')

    congestion_values = []
    for idx in range(start_idx, end_idx + 1):
        station = stations[idx]
        input_data = pd.DataFrame({
            'YEAR': [input_YEAR],
            'MONTH': [input_MONTH],
            'DAY': [input_DAY],
            'STATION': [station],
            'DIRECTION': [direction],
            'TIME_00': [input_TIME]
        })
        
        prediction = model.predict(input_data)
        # print(f"역: {station}, 예측 혼잡도: {prediction[0]:.2f}")
        congestion_values.append(prediction[0])

    avg_congestion = compute_average_congestion(congestion_values)
    return avg_congestion
    # print(f"평균 혼잡도: {avg_congestion:.2f}")

In [6]:
print_congestion_predictions('시청', '강남', 2023, 5, '월', 2, 'TIME_18', list_data)

역: 시청, 예측 혼잡도: 68.94
역: 을지로입구, 예측 혼잡도: 75.82
역: 을지로3가, 예측 혼잡도: 77.15
역: 을지로4가, 예측 혼잡도: 81.48
역: 동대문역사문화공원, 예측 혼잡도: 76.30
역: 신당, 예측 혼잡도: 75.42
역: 상왕십리, 예측 혼잡도: 78.19
역: 왕십리, 예측 혼잡도: 60.95
역: 한양대, 예측 혼잡도: 78.82
역: 뚝섬, 예측 혼잡도: 64.71
역: 성수, 예측 혼잡도: 53.88
역: 건대입구, 예측 혼잡도: 54.82
역: 구의, 예측 혼잡도: 57.26
역: 강변, 예측 혼잡도: 48.44
역: 잠실나루, 예측 혼잡도: 52.90
역: 잠실, 예측 혼잡도: 52.65
역: 신천, 예측 혼잡도: 74.38
역: 종합운동장, 예측 혼잡도: 47.41
역: 삼성, 예측 혼잡도: 82.26
역: 선릉, 예측 혼잡도: 124.54
역: 역삼, 예측 혼잡도: 142.65
역: 강남, 예측 혼잡도: 133.54
평균 혼잡도: 75.57
